# Predicting the intensity of the magnetic field experienced by satellites in Earth orbit

## Objective

To determine if the magnetic field experienced by satellites can be predicted from their altitude from Earth.

## Hypothesis

We can reasonably assume that the magnetic field will be less intense as the altitude increases.  However, because the Earth's geomagnetic field is not perfectly spherical but instead in the shape of a dipole, with anomalies and distortions from the pressure of the interplanetary magnetic field, the relationship between these two attributes might not be easily modeled.

## Dataset

MOST-268_HD209458_2014-268_HD209458_2014

A .tar file containing .fits files compressed as .tar files.

This dataset is available online: https://www.cadc-ccda.hia-iha.nrc-cnrc.gc.ca/en/search/?Plane.position.bounds@Shape1Resolver.value=ALL&Observation.collection=MOST&Observation.instrument.name=Direct%20image&Observation.type=object#sortCol=caom2%3APlane.time.bounds.lower&sortDir=dsc&col_1=_checkbox_selector;;;&col_2=caom2%3AObservation.uri;;;&col_3=caom2%3APlane.productID;;;&col_4=caom2%3AObservation.target.name;;;&col_5=caom2%3APlane.position.bounds.cval1;;;&col_6=caom2%3APlane.position.bounds.cval2;;;&col_7=caom2%3APlane.time.bounds.lower;;;&col_8=caom2%3AObservation.instrument.name;;;&col_9=caom2%3APlane.time.exposure;;;&col_10=caom2%3AObservation.proposal.pi;;;&col_11=caom2%3AObservation.proposal.id;;;&col_12=caom2%3APlane.calibrationLevel;;;&col_13=caom2%3AObservation.observationID;;;

The data was recorded by the MOST satellite: http://www.asc-csa.gc.ca/fra/satellites/most/default.asp

### Data Collection

In [ ]:
from cadcdata import StorageInventoryClient
client = StorageInventoryClient()

# test connection
print(client.cadcinfo("MOST/261_GSC0041702592_2014_5261.tar"))

In [ ]:
# download data as *.tar files

# download one file only for tests
# client.cadcget("MOST/261_GSC0041702592_2014_5261.tar", "./datasets/most/")

# download a list of files from a web search
with open("./datasets/most/cadcUrlList_test.txt", "r") as to_download:
    for row in to_download:
        f = row.split("cadc:")[1]
        client.cadcget(f, "./datasets/most/")

In [ ]:
import os
import tarfile

import pandas as pd
from astropy.io import fits


df_list = list()
df = pd.DataFrame(
    columns=[
        "[DEGREES] LONGITUDE OF SATELLITE",
        "[DEGREES] LATITUDE OF SATELLITE",
        "[M] ALTITUDE OF SATELLITE",
        "[DEGREES] ANGLE TO EARTH LIMB",
        "[DEGREES] NADIR RIGHT ASCENSION",
        "[DEGREES] NADIR DECLINATION",
        "[DEGREES] NADIR LONGITUDE",
        "[DEGREES] NADIR LATITUDE",
        "[DEGREES] SOLAR RIGHT ASCENSION",
        "[DEGREES] SOLAR DECLINATION",
        "[DEGREES] SOLAR ALTITUDE",
        "[DEGREES] SOLAR AZIMUTH",
        "[DEGREES] SOLAR LONGITUDE",
        "[DEGREES] SOLAR LATITUDE",
        "[DEGREES] LUNAR RIGHT ASCENSION",
        "[DEGREES] LUNAR DECLINATION",
        "[DEGREES] LUNAR ALTITUDE",
        "[DEGREES] LUNAR AZIMUTH",
        "[DEGREES] LUNAR LONGITUDE",
        "[DEGREES] LUNAR LATITUDE",
        "[DEGREES] LUNAR-TARGET ANGULAR SEPERATION",
        "[nT] MAGNETIC FIELD STRENGTH",
    ]
)


for root, dirs, files in os.walk("./datasets/most/"):
    for f in files:
        
        # for every .tar file in the datasets directory
        if os.path.splitext(f)[1] == ".tar":

            with tarfile.open(
                name=os.path.join(root, f),
                mode="r"
            ) as tar_obj:

                # for every file in the tar file
                for member in tar_obj.getnames():
                    if os.path.splitext(member)[1] == ".fits":

                        # extract .tar file in memory
                        extracted = tar_obj.extractfile(member)

                        # open extracted .fits file
                        with fits.open(extracted) as hdul:
                            hdr = hdul[0].header
                            data_dct = {
                                "[DEGREES] LONGITUDE OF SATELLITE": hdr["SAT_LONG"],
                                "[DEGREES] LATITUDE OF SATELLITE": hdr["SAT_LAT"],
                                "[M] ALTITUDE OF SATELLITE": hdr["SAT_ALT"],
                                "[DEGREES] ANGLE TO EARTH LIMB": hdr["ELA_ANG"],
                                "[DEGREES] NADIR RIGHT ASCENSION": hdr["NAD_RA"],
                                "[DEGREES] NADIR DECLINATION": hdr["NAD_DEC"],
                                "[DEGREES] NADIR LONGITUDE": hdr["NAD_PHI"],
                                "[DEGREES] NADIR LATITUDE": hdr["NAD_THET"],
                                "[DEGREES] SOLAR RIGHT ASCENSION": hdr["SOL_RA"],
                                "[DEGREES] SOLAR DECLINATION": hdr["SOL_DEC"],
                                "[DEGREES] SOLAR ALTITUDE": ["SOL_ALTI"],
                                "[DEGREES] SOLAR AZIMUTH": ["SOL_AZIM"],
                                "[DEGREES] SOLAR LONGITUDE": ["SOL_PHI"],
                                "[DEGREES] SOLAR LATITUDE": hdr["SOL_THET"],
                                "[DEGREES] LUNAR RIGHT ASCENSION": hdr["LUN_RA"],
                                "[DEGREES] LUNAR DECLINATION": hdr["LUN_DEC"],
                                "[DEGREES] LUNAR ALTITUDE": hdr["LUN_ALTI"],
                                "[DEGREES] LUNAR AZIMUTH": hdr["LUN_AZIM"],
                                "[DEGREES] LUNAR LONGITUDE": hdr["LUN_PHI"],
                                "[DEGREES] LUNAR LATITUDE": hdr["LUN_THET"],
                                "[DEGREES] LUNAR-TARGET ANGULAR SEPERATION": hdr["LUN_SEP"],
                                "[nT] MAGNETIC FIELD STRENGTH": hdr["MAG_FLD"],
                            }
                            df_partial = pd.DataFrame(data_dct)
                            df_list.append(df_partial)

In [ ]:
df_source = pd.concat(df_list, ignore_index=True)

### Exploratory Data Analysis

In [ ]:
df_source.head()

In [ ]:
df_source.tail()

In [ ]:
df_source.info()

In [ ]:
df_source.describe()

In [ ]:
# examine cardinality

df_source.nunique()

In [ ]:
df_source.isna().sum()

### Data Cleaning and Pre-Processing

In [ ]:
# impute missing values
from sklearn.impute import SimpleImputer

my_imputer = SimpleImputer(strategy="most_frequent", add_indicator=True, copy=True)
df_unscaled = pd.DataFrame(my_imputer.fit_transform(df_source))

In [ ]:
df_unscaled.columns = df_source.columns
df_unscaled.head()

In [ ]:
# change tabular into numeric


from category_encoders import OneHotEncoder
import numpy as np

encoder = OneHotEncoder(
    cols=[
        "[DEGREES] SOLAR ALTITUDE",
        "[DEGREES] SOLAR AZIMUTH",
        "[DEGREES] SOLAR LONGITUDE",
    ],
    handle_unknown='return_nan',
    return_df=True,
    use_cat_names=True,
)

df_unscaled_numeric = encoder.fit_transform(df_unscaled)
df = df_unscaled_numeric.astype(np.float64)

In [ ]:
# data normalization - https://datascience.stackexchange.com/a/13575

#from sklearn import preprocessing

#scaler = preprocessing.StandardScaler()
#df = pd.DataFrame(
#    scaler.fit_transform(df_unscaled_numeric_dtypes), 
#    columns=df_unscaled_numeric_dtypes.columns
#)

In [ ]:
df.info()

In [ ]:
df.head()

### Baseline Model training and validation

In [ ]:
# Identifier les variables dépendantes et indépendantes
df_target = df["[nT] MAGNETIC FIELD STRENGTH"]
df_predictors = df.drop(["[nT] MAGNETIC FIELD STRENGTH"], axis=1)

In [ ]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import train_test_split

# Séparer le jeu de données en jeu de donnée d'entraînement et de test
X_train, X_test, y_train, y_test = train_test_split(
    df_predictors, 
    df_target,
    train_size=0.8, 
    test_size=0.2, 
    random_state=10,
)

In [ ]:
# Fonction permettant d'évaluer le MAE d'un randomForestRegressor
def score_dataset(X_train, X_test, y_train, y_test):
    model = RandomForestRegressor()
    model.fit(X_train, y_train)
    preds = model.predict(X_test)
    return mean_absolute_error(y_test, preds)